In [3]:
import credentials # Import api/access_token keys from credentials.py
import re
import tweepy
import settings
import mysql.connector
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import preprocessor as p

# p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor')

class MyStreamListener(tweepy.StreamListener):

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    def lemma(comment):
        doc = MyStreamListener.nlp(comment)
        return " ".join([token.lemma_ for token in doc])    

    def deEmojify(self, text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def preprocess(self, tweet):    
        text = re.sub(r'http\S+', '', tweet.lower(), flags=re.MULTILINE)
        res = re.sub(r'[^\w\s]', '', text)
        deemo = self.deEmojify(res)
        le = self.lemma(deemo)
        return le

    def on_status(self, status):
        if status.retweeted:
        #Avoid retweeted info, and only original tweets will be received
            return True
        # Extract info from tweets
        id_str = status.id_str
        created_at = status.created_at
        # text = self.preprocess(status.text)    # Pre-processing the text  
        text = p.clean(status.text)
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        doc = nlp(text)
        enti = str()
        cond = ['PERSON', 'GPE', 'ORG']
        for ent in doc.ents: 
            if ent.label_ in cond:
                enti = enti + "," + ent.text                 
        enti = enti[1:]
        user_created_at = status.user.created_at
        user_location = self.deEmojify(status.user.location)
        user_description = self.deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
        
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print(text)
        print("Long: {}, Lati: {}".format(longitude, latitude))

        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, named_ent, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, enti, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()


    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, 
        stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
# nltk.download('stopwords')

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(tweet):    
    text = re.sub(r'http\S+', '', tweet.lower(), flags=re.MULTILINE)
    print(text)
    res = re.sub(r'[^\w\s]', '', text)
    print(res)
    deemo = deEmojify(res)
    print(deemo)
    le = lemma(deemo)
    print(le)
    return le

def lemma(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])    

In [3]:
print(lemma("Hello can't you do this?"))

hello ca n't you do this ?


In [59]:
print(preprocess("Hello Parth #happy"))

hello parth #happy
hello parth happy
hello parth happy
hello parth happy
hello parth happy


In [5]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB"
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

In [13]:
from sqlite3 import OperationalError

def check_conn(conn):
    try:
        conn.cursor()
        return True
    except Exception as ex:
        return False

myconn =sqlite3.connect('twitter.db')

if check_conn(myconn) == True:        
    mycursor = myconn.cursor()
    try:
        output=myconn.execute("SELECT COUNT(*) FROM {}".format(settings.TABLE_NAME))
        result=output.fetchall()
    except OperationalError:
        myconn.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        myconn.commit()

    myconn.close()

In [ ]:
auth = tweepy.OAuthHandler(credentials.consumer_key credentials.consumer_secret)
auth.set_access_token(credentials.access_token, credentials.access_token_secret)    
api = tweepy.API(auth,wait_on_rate_limit=True)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
myconn.close()

In [8]:
import spacy

nlp = spacy.load('en_core_web_sm') 

sentence = "Hello, my name is Parth and I live in Mumbai. I love eating cake and I was in Madrid last week. I use Microsoft's Laptop. Audi is a company that will grow a lot."

doc = nlp(sentence)

enti = str()
cond = ['PERSON', 'GPE', 'ORG']
for ent in doc.ents: 
	if ent.label_ in cond:
		enti = enti + "," + ent.text
		print(ent.text, ent.start_char, ent.end_char, ent.label_) 

enti = enti[1:]
print(enti)

Parth 18 23 PERSON
Mumbai 38 44 GPE
Madrid 78 84 GPE
Microsoft 102 111 ORG
Audi 122 126 ORG
Parth,Mumbai,Madrid,Microsoft,Audi


In [16]:
import re
text = re.findall("@([a-zA-Z0-9_]+)", "https://t.co/0zyatHjUMa  @ImranKhanPTI @PTIofficial @pmln_org  question for all of you regarding taxes and level of collection")
print(text)



['ImranKhanPTI', 'PTIofficial', 'pmln_org']


In [5]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os

java_path = "C:/Program Files/Java/jdk1.8.0_131/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('D:/Parth/Downloads/Stanford NER files/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz',
					   'D:/Parth/Downloads/Stanford NER files/stanford-ner-2020-11-17/stanford-ner.jar',
					   encoding='utf-8')

text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)


print(classified_text[2][1])

LOCATION


In [1]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapiExercises")
try:
    location = geolocator.geocode("Canajoharie, NY")
except Exception as e:
    print(e)
hello=location.raw['display_name'].split(",")
print(hello[len(hello)-1])

# print((location.latitude, location.longitude))

 United States


In [ ]:
'''
Further steps:

b) Convert country names to country codes
2) Calculate the total number of occurences of country 
3) Combine all the data into a DataFrame and plot
'''

In [17]:
import nltk
# nltk.download('words')
words = set(nltk.corpus.words.words())
flag=0
for i in words:
    if 'dear' in i.lower():
        flag=1
if flag==1:
    print("YES")
else:
    print("no")
# t="Österreich"
# if t=="Österreich":
#     print("Austria")

YES


In [14]:
import country_converter as coco
cc = coco.CountryConverter()

some_names = ['United Rep. of Tanzania', 'Cape Verde', 'Polska',
              'Iran (Islamic Republic of)', 'Korea, Republic of',
              "Dem. People's Rep. of Korea"]

standard_names = cc.convert(names = some_names, to = 'iso2')
UNmembership = cc.convert(names = some_names, to = 'UNmember')
print(standard_names)
print(UNmembership)

Polska not found in regex
Polska not found in regex
['TZ', 'CV', 'not found', 'IR', 'KR', 'KP']
[1961, 1975, 'not found', 1945, 1991, 1991]


In [51]:
hello='Italia'
country={'Italia':'Italy', 'Việt Nam':'Vietnam', 'Ayiti':'Haiti','Österreich':'Austria', 'Česko':'Czechia', 'Sverige':'Sweden', 'Luzon': 'Philippines'}
if hello in country.keys():
    print(country[hello])

Italy


In [53]:
from langdetect import detect
detect("日本")

'zh-cn'

In [28]:
import pycountry
import gettext
user_input = "Frankreich"
country_code = ''.join([country.alpha2 for country in pycountry.countries if _(country.name) == user_input]) # u'FR'
print(country_code)

TypeError: 'str' object is not callable

In [38]:
import pycountry
import gettext

def map_country_code(country_name, language):
    try:
        # todo:  when languages = en the program fails!
        # todo: this methode takes to much time, is it o(n2) have to think on a better system+
        if country_name is None:
            return None
        german = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=[language])
        german.install()
        _ = german.gettext
        for english_country in pycountry.countries:
            country_name = country_name.lower()
            german_country = _(english_country.name).lower()
            if german_country == country_name:
                return english_country.alpha_3
    except Exception as e:
        self.log.error(e, exc_info=True)
        return False

print(map_country_code("Česko", 'de'))

None


In [65]:
# from googletrans import Translator
# translator = Translator()
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('Ayiti',lang_tgt='en')  
print(translate_text)
# dt1 = translator.detect('ليبيا')
# print(dt1)


Haiti 


In [1]:
import os
path='D:/Parth/Downloads/train'
for (dirpath,dirnames,filenames) in os.walk(path):
    for dirname in dirnames:
        print(dirname)
        for(direcpath,direcnames,files) in os.walk(path+"\\"+dirname):
            for file in files:
                actual_path=path+"\\\\"+dirname+"\\\\"+file
                print(actual_path)
                des=func(actual_path)
                img_descs.append(des)
                y.append(label)
        label=label+1

A
D:/Parth/Downloads/train\\A\\001.jpg


NameError: name 'func' is not defined

In [6]:
from google_trans_new import google_translator 
from geopy.geocoders import Nominatim
import country_converter as coco

def get_location(temp_location):
    geolocator = Nominatim(user_agent="myGeocoder")         # Initializing geolocator object for getting address
    try:
        location = geolocator.geocode(temp_location)
        # print(location.raw['address']['country'])                
        location_list = location.raw['display_name'].split(",")
        user_location1=location_list[len(location_list)-1].strip()               # Extracting only country name
        translator = google_translator()  
        user_location = translator.translate(user_location1, lang_tgt='en')         #Translate the location to english
    except Exception as e:
        user_location = None

    return user_location

print(get_location('Luzon'))


iso2_codes = coco.convert(names=get_location('Luzon'), to='ISO3')
print(iso2_codes)
# counter=collections.Counter(iso2_codes)

# for i in df1['CODE']:
hello=coco.convert(names = iso2_codes, to = 'name_short')    
print(hello)


Philippines 
PHL
Philippines
